In [ ]:
!pip install ultralytics pandas scikit-learn

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/78.2 GB disk)


In [ ]:
!unzip ppe_dataset.zip
from IPython import display
display.clear_output()

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from ultralytics import YOLO

In [ ]:
DATASET_PATH = '/content/ppe_dataset/'
IMG_DIR = os.path.join(DATASET_PATH, 'images')
LABEL_DIR = os.path.join(DATASET_PATH, 'labels')

In [ ]:
def load_dataset_info(img_dir, label_dir):
    image_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
    data = []
    for img_file in image_files:
        label_file = os.path.splitext(img_file)[0] + '.txt'
        if os.path.exists(os.path.join(label_dir, label_file)):
            with open(os.path.join(label_dir, label_file), 'r') as f:
                labels = [line.strip().split() for line in f.readlines()]
                for label in labels:
                    class_id = int(label[0])
                    data.append((img_file, class_id))
    return pd.DataFrame(data, columns=['image', 'class_id'])

dataset_info = load_dataset_info(IMG_DIR, LABEL_DIR)

In [ ]:
# Stratified K-Fold Cross-Validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(dataset_info['image'], dataset_info['class_id'])):
    print(f"Training fold {fold + 1}/{n_splits}...")

    # Create train and validation datasets
    train_data = dataset_info.iloc[train_idx]
    val_data = dataset_info.iloc[val_idx]

    # Save train and validation data to text files
    with open('/content/train.txt', 'w') as f:
        for _, row in train_data.iterrows():
            f.write(f"{os.path.join(IMG_DIR, row['image'])}\n")

    with open('/content/val.txt', 'w') as f:
        for _, row in val_data.iterrows():
            f.write(f"{os.path.join(IMG_DIR, row['image'])}\n")

    # Create a data.yaml file for YOLOv8
    with open('/content/data.yaml', 'w') as f:
        f.write(f"train: /content/train.txt\n")
        f.write(f"val: /content/val.txt\n")
        f.write(f"nc: 7\n")  # Number of classes (adjust as needed)
        f.write(f"names: ['hard-hat', 'gloves', 'mask', 'glasses', 'boots', 'vest', 'ppe-suit']\n")

    model = YOLO('yolov8s.pt')

    # Fine-tune the model
    model.train(data='/content/data.yaml',
                epochs=50,
                batch=32,
                imgsz=640)

    # Evaluate the model
    results = model.val(data='/content/data.yaml')
    print(f"Fold {fold + 1} results: {results}")

print("Training completed for all folds.")

Training fold 1/5...


100%|██████████| 21.5M/21.5M [00:00<00:00, 117MB/s] 


Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/data.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

100%|██████████| 755k/755k [00:00<00:00, 24.3MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...


50 epochs completed in 0.413 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 22.5MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:10<00:00,  2.17s/it]


                   all        294        702      0.967      0.943      0.966       0.93
              hard-hat        260        295      0.953      0.922      0.966      0.897
                gloves        160        259      0.988      0.927      0.957      0.911
                  mask          9          9      0.987          1      0.995      0.985
                  vest         70         73      0.971      0.959      0.964      0.926
              ppe-suit         62         66      0.935      0.909      0.947      0.933
Speed: 0.2ms preprocess, 4.0ms inference, 0.0ms loss, 6.7ms postprocess per image
Results saved to runs/detect/train6
Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs


val: Scanning /content/ppe_dataset/labels.cache... 294 images, 0 backgrounds, 0 corrupt: 100%|██████████| 294/294 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:09<00:00,  1.07it/s]


                   all        294        702      0.967      0.943      0.966       0.93
              hard-hat        260        295      0.954      0.922      0.963      0.897
                gloves        160        259      0.987      0.927      0.956      0.909
                  mask          9          9      0.987          1      0.995      0.985
                  vest         70         73      0.971      0.959      0.963      0.921
              ppe-suit         62         66      0.934      0.909      0.952      0.939
Speed: 0.3ms preprocess, 6.9ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to runs/detect/train62
Fold 5 results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d0dda555d20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(

In [ ]:
import shutil
from google.colab import files

# Zip the directory
shutil.make_archive('/content/runs', 'zip', '/content/runs')

# Download the zipped file
files.download('runs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>